**Бутстреп**  - метод оценки доверительного интервала cтатистики, основанный на многократной генерации случайных подвыборок из имеющихся данных.

Бутстрап нужен когда нужно сравнить не средние у 2 выборок или требования к Т-тесту не выполнены (он не работает)

Но бутстрап грешит высокими затратами времени и ресурсов на больших выборках, поэтому его не приоритетно применять. Т-тест всегда быстрее.

**Порядок действий:**

Исходные данные - две выборки (А и Б)

0. Считаем реальную разницу статистик (например разницу средних, 95, 99 квантиля выборок) ***real_diff***

1. На каждой итерации генерируем 2 синтетические подвыборки и считаем разницу получившихся статистик (например разницу средних, 95, 99 квантиля выборок), записываем в список

2. Повторяем итерации 100-10000 раз
3. По полученному списку строим доверительный интервал с помощью квантилей.
4. Проверям чтоб в полученный интервал не входил 0. Если 0 входит - то не можем отвергнуть нулевую гипотезу и разница двух выборок не стат. значима.
5. Если 0 не входит то все хорошо и можем утверждать, что "Выборки стат. значимо отличаются на ***real_diff*** и разница с вероятностью p% лежит в интервале ..."

Роль p-value тут играет вероятность доверительного интервала (alpha=0.05 соответствует 95%)

# Пример (сравнение средних двух выборок):

In [7]:
import numpy as np
import pandas as pd

#### создаем 2 рандомные выборки (для примера):

In [78]:
sample_1 = np.random.normal(10, 20, 1000)
sample_2 = np.random.normal(15, 20, 1000)

#### пример сравнения разницы средних двух выборок:

In [79]:
bs_diffs = []

for i in range(10_000):
    bs_1 = np.random.choice(sample_1, size=len(sample_1), replace=True)
    bs_2 = np.random.choice(sample_2, size=len(sample_1), replace=True)
    bs_diff = np.mean(bs_1) - np.mean(bs_2)
    bs_diffs.append(bs_diff)  

#### оцениваем доверит. интервал для разницы между средними в выборках:

In [75]:
perc = 95 # вероятность интервала (стат. значимость alpha=0.05 => 95%)

lower_q = (1-perc/100)/2 # нижний квантиль
upper_q = 1 - (1-perc/100)/2 # верхний квантиль

# получаем границы интервала:
lower_bound = np.quantile(bs_diffs, lower_q)
upper_bound = np.quantile(bs_diffs, upper_q)

# считаем реальную разницу средних между выборками:
real_diff = np.mean(sample_1) - np.mean(sample_2)

# проверяем входит ли ноль в интервал (если входит то не можем говорить о стат. значимых различиях):
if lower_bound <= 0 <= upper_bound:
    print("Не можем отвергнуть нулевую гипотезу. Нет стат. значимого различия между средними в выборках.")
else:
    print(f'Выборки стат. значимо отличаются на {real_diff:.2f}. Разница лежит с вероятностью {perc} % - в интервале {lower_bound:.2f}..{upper_bound:.2f}')

Выборки стат. значимо отличаются на -5.21. Разница лежит с вероятностью 95 % - в интервале -6.77..-3.48


# Проверим, дает ли бутстрап с установленным 95%-интервалом ошибку первого рода = 0.05:

In [81]:
import numpy as np

# ф-ция проведения бутстрапа:
def bootstrap_test(sample_1, sample_2, n_bootstraps=1000, perc=95):
    bs_diffs = []

    for i in range(n_bootstraps):
        bs_1 = np.random.choice(sample_1, size=len(sample_1), replace=True)
        bs_2 = np.random.choice(sample_2, size=len(sample_2), replace=True)
        bs_diff = np.mean(bs_1) - np.mean(bs_2)
        bs_diffs.append(bs_diff)

    lower_q = (1 - perc / 100) / 2
    upper_q = 1 - (1 - perc / 100) / 2

    lower_bound = np.quantile(bs_diffs, lower_q)
    upper_bound = np.quantile(bs_diffs, upper_q)

    return lower_bound, upper_bound

# Параметры теста
n_tests = 1000  # Количество тестов
sample_size = 1000  # Размер каждой выборки
true_mean = 10  # Истинное среднее значение выборок
std_dev = 20  # Стандартное отклонение выборок
alpha = 0.05  # Уровень значимости

type_1_errors = 0

for _ in range(n_tests):
    # Генерируем две выборки с одинаковыми средними
    sample_1 = np.random.normal(true_mean, std_dev, sample_size)
    sample_2 = np.random.normal(true_mean, std_dev, sample_size)
    
    # Выполняем бутстреп-тест
    lower_bound, upper_bound = bootstrap_test(sample_1, sample_2)
    
    # Проверяем, отвергается ли нулевая гипотеза
    if lower_bound > 0 or upper_bound < 0:
        type_1_errors += 1

# Вероятность ошибки первого рода
type_1_error_rate = type_1_errors / n_tests
print(f'Вероятность ошибки первого рода: {type_1_error_rate:.3f}')

Вероятность ошибки первого рода: 0.053
